# 📊 SQL-Python Analysis Example: Customer Segmentation

**Business Question**: How can we segment our customers to improve targeted marketing?

**Analyst Goal**: Use SQL to extract customer data, then Python to perform RFM analysis and create actionable customer segments.

**Tools Used**: SQL for data extraction, pandas for analysis, matplotlib for visualization

## Setup and Database Connection

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy as sa
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Database connection setup
# Replace with your actual database credentials
connection_string = "sqlite:///sample_database/ecommerce.db"  # Using SQLite for demo

try:
    engine = sa.create_engine(connection_string)
    
    # Test connection
    with engine.connect() as conn:
        test_query = conn.execute(sa.text("SELECT 1 as connection_test"))
        result = test_query.fetchone()
        print(f"✅ Database connection successful: {result[0]}")
        
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("Note: Make sure your database is running and credentials are correct")

## Step 1: Data Extraction with SQL

We'll use SQL to calculate the core RFM metrics:
- **Recency**: Days since last purchase
- **Frequency**: Number of orders
- **Monetary**: Total amount spent

In [ ]:
# SQL query to calculate RFM metrics
rfm_query = """
WITH customer_metrics AS (
    SELECT 
        customer_id,
        MAX(order_date) as last_order_date,
        COUNT(DISTINCT order_id) as frequency,
        SUM(order_total) as monetary_value,
        AVG(order_total) as avg_order_value
    FROM orders 
    WHERE order_date >= DATE('now', '-2 years')  -- Last 2 years of data
        AND order_status = 'completed'
    GROUP BY customer_id
),
rfm_calculations AS (
    SELECT 
        cm.*,
        -- Calculate recency (days since last order)
        CAST(julianday('now') - julianday(cm.last_order_date) AS INTEGER) as recency_days,
        
        -- Add customer info
        c.customer_name,
        c.email,
        c.acquisition_date,
        c.customer_segment as current_segment
        
    FROM customer_metrics cm
    JOIN customers c ON cm.customer_id = c.customer_id
    WHERE cm.frequency >= 1  -- At least one order
)
SELECT * FROM rfm_calculations
ORDER BY monetary_value DESC
"""

print("Executing RFM analysis query...")
rfm_data = pd.read_sql_query(rfm_query, engine)

print(f"✅ Data extracted: {len(rfm_data):,} customers")
print(f"Date range: {rfm_data['last_order_date'].min()} to {rfm_data['last_order_date'].max()}")
rfm_data.head()

## Step 2: Data Exploration and Quality Check

In [ ]:
# Basic data quality and exploration
print("📊 RFM Data Overview")
print("=" * 50)
print(f"Total customers: {len(rfm_data):,}")
print(f"Missing values: {rfm_data.isnull().sum().sum()}")
print("\nRFM Statistics:")
print(rfm_data[['recency_days', 'frequency', 'monetary_value']].describe())

In [ ]:
# Visualize the distribution of RFM metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('RFM Metrics Distribution', fontsize=16, fontweight='bold')

# Recency distribution
axes[0,0].hist(rfm_data['recency_days'], bins=30, edgecolor='black', alpha=0.7)
axes[0,0].set_title('Recency (Days Since Last Order)')
axes[0,0].set_xlabel('Days')
axes[0,0].set_ylabel('Number of Customers')

# Frequency distribution
axes[0,1].hist(rfm_data['frequency'], bins=20, edgecolor='black', alpha=0.7)
axes[0,1].set_title('Frequency (Number of Orders)')
axes[0,1].set_xlabel('Orders')
axes[0,1].set_ylabel('Number of Customers')

# Monetary distribution (log scale for better visualization)
axes[1,0].hist(np.log1p(rfm_data['monetary_value']), bins=30, edgecolor='black', alpha=0.7)
axes[1,0].set_title('Monetary Value (Log Scale)')
axes[1,0].set_xlabel('Log(Total Spent + 1)')
axes[1,0].set_ylabel('Number of Customers')

# Average order value
axes[1,1].hist(rfm_data['avg_order_value'], bins=30, edgecolor='black', alpha=0.7)
axes[1,1].set_title('Average Order Value')
axes[1,1].set_xlabel('Average Order Value ($)')
axes[1,1].set_ylabel('Number of Customers')

plt.tight_layout()
plt.show()

## Step 3: RFM Scoring and Segmentation

Now we'll use Python to create RFM scores and customer segments.

In [ ]:
# Create RFM scores using quintiles
def calculate_rfm_scores(df):
    """Calculate RFM scores using quintile-based scoring"""
    
    # Calculate quintiles for each metric
    # Note: For recency, lower values (more recent) get higher scores
    df['r_score'] = pd.qcut(df['recency_days'], q=5, labels=[5,4,3,2,1])
    df['f_score'] = pd.qcut(df['frequency'].rank(method='first'), q=5, labels=[1,2,3,4,5])
    df['m_score'] = pd.qcut(df['monetary_value'].rank(method='first'), q=5, labels=[1,2,3,4,5])
    
    # Convert to numeric
    df['r_score'] = df['r_score'].astype(int)
    df['f_score'] = df['f_score'].astype(int)
    df['m_score'] = df['m_score'].astype(int)
    
    # Create combined RFM score
    df['rfm_score'] = df['r_score'].astype(str) + df['f_score'].astype(str) + df['m_score'].astype(str)
    
    return df

# Apply RFM scoring
rfm_scored = calculate_rfm_scores(rfm_data.copy())

print("✅ RFM scores calculated")
print("\nSample RFM scores:")
print(rfm_scored[['customer_id', 'customer_name', 'recency_days', 'frequency', 'monetary_value', 
                  'r_score', 'f_score', 'm_score', 'rfm_score']].head(10))

In [ ]:
# Define customer segments based on RFM scores
def segment_customers(df):
    """Assign customer segments based on RFM scores"""
    
    def rfm_segment(row):
        r, f, m = row['r_score'], row['f_score'], row['m_score']
        
        # Champions: High recency, frequency, and monetary
        if r >= 4 and f >= 4 and m >= 4:
            return 'Champions'
        
        # Loyal Customers: High frequency and monetary, varying recency
        elif f >= 4 and m >= 4:
            return 'Loyal Customers'
        
        # Potential Loyalists: Recent customers with good frequency
        elif r >= 4 and f >= 3:
            return 'Potential Loyalists'
        
        # Recent Customers: High recency, low frequency
        elif r >= 4 and f <= 2:
            return 'Recent Customers'
        
        # Promising: Recent with moderate frequency and monetary
        elif r >= 3 and f >= 2 and m >= 2:
            return 'Promising'
        
        # Customers Needing Attention: Moderate across all metrics
        elif r >= 2 and f >= 2 and m >= 2:
            return 'Customers Needing Attention'
        
        # About to Sleep: Low recency but decent frequency and monetary
        elif r <= 2 and f >= 3 and m >= 3:
            return 'About to Sleep'
        
        # At Risk: Low recency and frequency, high monetary
        elif r <= 2 and f <= 2 and m >= 4:
            return 'At Risk'
        
        # Cannot Lose Them: Low recency but very high monetary
        elif r <= 2 and m >= 5:
            return 'Cannot Lose Them'
        
        # Hibernating: Low across all metrics but not lost
        elif r <= 2 and f <= 2 and m <= 3:
            return 'Hibernating'
        
        # Lost: Very low across all metrics
        else:
            return 'Lost'
    
    df['customer_segment'] = df.apply(rfm_segment, axis=1)
    return df

# Apply segmentation
rfm_segmented = segment_customers(rfm_scored.copy())

print("✅ Customer segmentation completed")
print("\nSegment distribution:")
segment_counts = rfm_segmented['customer_segment'].value_counts()
print(segment_counts)
print(f"\nTotal customers segmented: {len(rfm_segmented):,}")

## Step 4: Segment Analysis and Business Insights

In [ ]:
# Detailed segment analysis
segment_analysis = rfm_segmented.groupby('customer_segment').agg({
    'customer_id': 'count',
    'recency_days': ['mean', 'median'],
    'frequency': ['mean', 'median'],
    'monetary_value': ['mean', 'median', 'sum'],
    'avg_order_value': ['mean', 'median']
}).round(2)

# Flatten column names
segment_analysis.columns = ['_'.join(col).strip() for col in segment_analysis.columns]
segment_analysis = segment_analysis.rename(columns={'customer_id_count': 'customer_count'})

# Calculate percentage of total customers and revenue
total_customers = len(rfm_segmented)
total_revenue = rfm_segmented['monetary_value'].sum()

segment_analysis['customer_percentage'] = (segment_analysis['customer_count'] / total_customers * 100).round(1)
segment_analysis['revenue_percentage'] = (segment_analysis['monetary_value_sum'] / total_revenue * 100).round(1)

# Sort by revenue contribution
segment_analysis = segment_analysis.sort_values('monetary_value_sum', ascending=False)

print("📊 Customer Segment Analysis")
print("=" * 80)
display(segment_analysis)

In [ ]:
# Visualization: Segment distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Customer Segment Analysis', fontsize=16, fontweight='bold')

# Customer count by segment
segment_counts.plot(kind='bar', ax=axes[0,0], color='skyblue', edgecolor='black')
axes[0,0].set_title('Number of Customers by Segment')
axes[0,0].set_ylabel('Number of Customers')
axes[0,0].tick_params(axis='x', rotation=45)

# Revenue by segment
revenue_by_segment = rfm_segmented.groupby('customer_segment')['monetary_value'].sum().sort_values(ascending=False)
revenue_by_segment.plot(kind='bar', ax=axes[0,1], color='lightcoral', edgecolor='black')
axes[0,1].set_title('Total Revenue by Segment')
axes[0,1].set_ylabel('Total Revenue ($)')
axes[0,1].tick_params(axis='x', rotation=45)

# Average order value by segment
aov_by_segment = rfm_segmented.groupby('customer_segment')['avg_order_value'].mean().sort_values(ascending=False)
aov_by_segment.plot(kind='bar', ax=axes[1,0], color='lightgreen', edgecolor='black')
axes[1,0].set_title('Average Order Value by Segment')
axes[1,0].set_ylabel('Average Order Value ($)')
axes[1,0].tick_params(axis='x', rotation=45)

# Segment pie chart (by revenue)
axes[1,1].pie(revenue_by_segment.values, labels=revenue_by_segment.index, autopct='%1.1f%%', startangle=90)
axes[1,1].set_title('Revenue Distribution by Segment')

plt.tight_layout()
plt.show()

## Step 5: Business Recommendations and Action Items

In [ ]:
# Generate actionable business recommendations
def generate_recommendations(segment_analysis, rfm_segmented):
    """Generate business recommendations based on segment analysis"""
    
    recommendations = {}
    
    # Get top segments by customer count and revenue
    top_revenue_segments = segment_analysis.nlargest(3, 'monetary_value_sum').index.tolist()
    top_customer_segments = segment_analysis.nlargest(3, 'customer_count').index.tolist()
    
    # Champions strategy
    if 'Champions' in rfm_segmented['customer_segment'].values:
        champions_count = len(rfm_segmented[rfm_segmented['customer_segment'] == 'Champions'])
        champions_revenue = rfm_segmented[rfm_segmented['customer_segment'] == 'Champions']['monetary_value'].sum()
        recommendations['Champions'] = {
            'strategy': 'Reward and Retain',
            'actions': [
                'Offer exclusive VIP programs and early access to new products',
                'Create referral incentives to leverage their advocacy',
                'Provide premium customer service and personalized experiences'
            ],
            'metrics': f"{champions_count:,} customers contributing ${champions_revenue:,.0f}"
        }
    
    # At Risk strategy
    if 'At Risk' in rfm_segmented['customer_segment'].values:
        at_risk_count = len(rfm_segmented[rfm_segmented['customer_segment'] == 'At Risk'])
        recommendations['At Risk'] = {
            'strategy': 'Win Back Campaign',
            'actions': [
                'Send personalized win-back emails with special offers',
                'Conduct survey to understand why they stopped purchasing',
                'Offer limited-time discounts or free shipping'
            ],
            'metrics': f"{at_risk_count:,} high-value customers at risk of churning"
        }
    
    # Recent Customers strategy
    if 'Recent Customers' in rfm_segmented['customer_segment'].values:
        recent_count = len(rfm_segmented[rfm_segmented['customer_segment'] == 'Recent Customers'])
        recommendations['Recent Customers'] = {
            'strategy': 'Nurture and Convert',
            'actions': [
                'Create onboarding email series with product education',
                'Offer new customer incentives for second purchase',
                'Provide excellent customer support to build trust'
            ],
            'metrics': f"{recent_count:,} new customers with conversion potential"
        }
    
    return recommendations

# Generate and display recommendations
recommendations = generate_recommendations(segment_analysis, rfm_segmented)

print("🎯 BUSINESS RECOMMENDATIONS")
print("=" * 80)

for segment, rec in recommendations.items():
    print(f"\n📊 {segment}")
    print(f"Strategy: {rec['strategy']}")
    print(f"Metrics: {rec['metrics']}")
    print("Recommended Actions:")
    for i, action in enumerate(rec['actions'], 1):
        print(f"  {i}. {action}")
    print("-" * 60)

## Step 6: Export Results for Business Use

In [ ]:
# Prepare data for export
export_data = rfm_segmented[[
    'customer_id', 'customer_name', 'email',
    'customer_segment', 'rfm_score',
    'recency_days', 'frequency', 'monetary_value', 'avg_order_value',
    'r_score', 'f_score', 'm_score',
    'last_order_date', 'acquisition_date'
]].copy()

# Add business priority flag
high_priority_segments = ['Champions', 'At Risk', 'Cannot Lose Them', 'Loyal Customers']
export_data['priority'] = export_data['customer_segment'].apply(
    lambda x: 'High' if x in high_priority_segments else 'Medium'
)

# Sort by priority and monetary value
export_data = export_data.sort_values(['priority', 'monetary_value'], ascending=[True, False])

print("📤 Preparing data for export...")
print(f"Total records: {len(export_data):,}")
print(f"High priority customers: {len(export_data[export_data['priority'] == 'High']):,}")

# Display sample of high priority customers
print("\n🔥 High Priority Customers (Sample):")
display(export_data[export_data['priority'] == 'High'].head(10))

In [ ]:
# Export to Excel with multiple sheets
try:
    with pd.ExcelWriter('customer_segmentation_analysis.xlsx', engine='openpyxl') as writer:
        
        # Main customer data
        export_data.to_excel(writer, sheet_name='Customer_Segments', index=False)
        
        # Segment summary
        segment_analysis.to_excel(writer, sheet_name='Segment_Analysis')
        
        # High priority customers only
        high_priority_customers = export_data[export_data['priority'] == 'High']
        high_priority_customers.to_excel(writer, sheet_name='High_Priority', index=False)
        
        # Champions for VIP program
        champions = export_data[export_data['customer_segment'] == 'Champions']
        if len(champions) > 0:
            champions.to_excel(writer, sheet_name='VIP_Champions', index=False)
        
        # At-risk customers for win-back campaign
        at_risk = export_data[export_data['customer_segment'] == 'At Risk']
        if len(at_risk) > 0:
            at_risk.to_excel(writer, sheet_name='Win_Back_Campaign', index=False)
    
    print("✅ Analysis exported successfully to 'customer_segmentation_analysis.xlsx'")
    print("\n📋 Excel file contains:")
    print("  • Customer_Segments: Complete customer data with segments")
    print("  • Segment_Analysis: Statistical summary by segment")
    print("  • High_Priority: Customers requiring immediate attention")
    print("  • VIP_Champions: Top customers for VIP programs")
    print("  • Win_Back_Campaign: At-risk customers for retention efforts")
    
except Exception as e:
    print(f"❌ Export failed: {e}")
    print("Make sure you have openpyxl installed: pip install openpyxl")

## Summary: Key Business Insights

### 📊 Analysis Results
- **Total Customers Analyzed**: {len(rfm_segmented):,}
- **Segments Identified**: {len(segment_counts)} distinct customer segments
- **High-Value Segments**: Champions, Loyal Customers, Cannot Lose Them
- **Action Required**: At Risk, About to Sleep, Hibernating customers

### 🎯 Business Impact
1. **Revenue Focus**: Top 20% of customers likely drive 80% of revenue
2. **Retention Priority**: Immediate action needed for at-risk high-value customers
3. **Growth Opportunity**: Recent customers represent conversion potential
4. **Marketing Efficiency**: Targeted campaigns vs. mass marketing

### 🚀 Next Steps
1. **Implement recommended actions** for each customer segment
2. **Set up automated monitoring** to track segment changes
3. **Measure campaign effectiveness** for each segment
4. **Refine segmentation** based on business results

### 💡 Analyst Workflow Demonstrated
- **SQL for data extraction**: Efficient aggregation and calculation
- **Python for analysis**: Complex segmentation logic and statistics
- **Visualization for insights**: Clear communication of findings
- **Export for action**: Business-ready deliverables

---

**This analysis combines the power of SQL and Python to deliver actionable business insights. The same approach can be applied to other business questions like product analysis, marketing attribution, or operational efficiency.**